<a href="https://colab.research.google.com/github/reckoning-machines/sec_text_nlp/blob/master/sec_nlp_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://gist.github.com/Joshua1989/dc7e60aa487430ea704a8cb3f2c5d6a6

# we need to reinitialize local folders and data.

!mkdir sec_data_folder

mkdir: cannot create directory ‘sec_data_folder’: File exists


In [2]:
!git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /tmp/colab_util_repo
!mv /tmp/colab_util_repo/colab_util.py colab_util.py 
!rm -r /tmp/colab_util_repo
from colab_util import *
drive_handler = GoogleDriveHandler()

Cloning into '/tmp/colab_util_repo'...
remote: Enumerating objects: 40, done.
remote: Total 40 (delta 0), reused 0 (delta 0), pack-reused 40
Unpacking objects: 100% (40/40), done.


In [0]:
#drive_handler.download('implementation_ticker_list.csv', target_path='implementation_ticker_list.csv')
#first pull the ticker list with google drive handler lib

#drive_handler.download('test_ticker_list.csv', target_path='test_ticker_list.csv')
drive_handler.download('implementation_ticker_list.csv', target_path='implementation_ticker_list.csv')

import pandas as pd
df_tickers = pd.read_csv('implementation_ticker_list.csv')
#df_tickers = df_tickers[df_tickers['Symbol']=='FRC']
#i = df_tickers[df_tickers['Symbol']=='TROW'].index[0]
#df_tickers = df_tickers.iloc[i:,:]
#df_tickers.to_csv('implementation_ticker_list.csv')
#df_tickers[df_tickers['Symbol'] == 'NWL'].index



In [4]:

import os.path
from os import path

str_id = ''
df_tickers = pd.read_csv('implementation_ticker_list.csv')

for i, row in df_tickers.iterrows():
  str_id = ''
  str_ticker = row['Symbol']
  print('pulling csv for:'+str_ticker+"...")
  
  str_to_file = 'sec_data_folder/'+str_ticker+'.csv'
  str_from_file = '/sec_data_folder/'+str_ticker+'.csv'

  if not path.exists(str_to_file):
    try:
      str_id = drive_handler.path_to_id('/sec_data_folder/'+str_ticker+'.csv')
    except:
      print('/sec_data_folder/'+str_ticker+'.csv not found...')
  
    if len(str_id) > 0:
      drive_handler.download(str_to_file, target_path=str_from_file)


pulling csv for:MMM...
pulling csv for:ABT...
pulling csv for:ABBV...
pulling csv for:ABMD...
pulling csv for:ACN...
pulling csv for:ATVI...
pulling csv for:ADBE...
pulling csv for:AMD...
pulling csv for:AAP...
pulling csv for:AES...
pulling csv for:AFL...
pulling csv for:A...
pulling csv for:APD...
pulling csv for:AKAM...
pulling csv for:ALK...
pulling csv for:ALB...
pulling csv for:ARE...
pulling csv for:ALXN...
pulling csv for:ALGN...
pulling csv for:ALLE...
pulling csv for:AGN...
pulling csv for:ADS...
pulling csv for:LNT...
pulling csv for:ALL...
pulling csv for:GOOG...
pulling csv for:MO...
pulling csv for:AMZN...
pulling csv for:AMCR...
pulling csv for:AEE...
pulling csv for:AAL...
pulling csv for:AEP...
pulling csv for:AXP...
pulling csv for:AIG...
pulling csv for:AMT...
pulling csv for:AWK...
pulling csv for:AMP...
pulling csv for:ABC...
pulling csv for:AME...
pulling csv for:AMGN...
pulling csv for:APH...
pulling csv for:ADI...
pulling csv for:ANSS...
pulling csv for:ANTM...


In [11]:
!pip install transformers
import torch
from transformers import *
#import academic standard NLP library nltk.  this will be our baseline.
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')
from transformers import pipeline
nlp = pipeline('sentiment-analysis')

import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

#replace with your own list of words ... like covid or delay or cancel (it lowercases automatically)
FIND_WORDS = ['covid',
              'guidance',
              'outlook']

def check_if_list_found_in_text(text, words=[], return_offset=False, lower_text=True):
    result = []
    text = (
        " "
        + text.replace("_", " ")
        .replace("-", " ")
        .replace(",", " ")
        .replace(";", " ")
        .replace('"', " ")
        .replace(":", " ")
        .replace(".", " ")
        + " "
    )
    if lower_text:
        text = text.lower()
    for word in words:
        word = (
            " "
            + word.replace("_", " ")
            .replace("-", " ")
            .replace(",", " ")
            .replace(";", " ")
            .replace('"', " ")
            .replace(":", " ")
            .replace(".", " ")
            + " "
        )
        if lower_text:
            word = word.lower()
        if word in text:
            if return_offset:
                offset = text.find(word)
                # offset = offset if not offset else offset-1
                result.append(offset)
            else:
                result.append(word.strip())
    return result

def filter_stopwords(sent):
    stop_words = set(stopwords.words('english'))   
    word_tokens = word_tokenize(sent)   
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    return ' '.join(filtered_sentence)

def sentiment_from_text(sentence):
#  dict_sentiment = {}
  sentence = filter_stopwords(sentence)
  list_found = check_if_list_found_in_text(sentence,FIND_WORDS)
  num_found = len(list_found)
  ###
  ss = sid.polarity_scores(sentence) #NLTK
  df = pd.DataFrame.from_dict(ss,orient = "index").T
  df['transformers_score'] = dict_transformers['score'] #tranformers
  df['transformers_label'] = dict_transformers['label']
  df['text'] = sentence
  df['keywords_found'] = num_found 
  return pd.concat(dict_sentiment)

  ###

def filter_stopwords(sent):
  stop_words = set(stopwords.words('english'))   
  word_tokens = word_tokenize(sent)   
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  filtered_sentence = [] 
  for w in word_tokens: 
      if w not in stop_words: 
          filtered_sentence.append(w)
  return ' '.join(filtered_sentence)

def df_from_text(text):
  sentence_list = tokenize.sent_tokenize(text)
  sentence_list
  sid = SentimentIntensityAnalyzer()
  list_df = []
  for sentence in sentence_list:
      sentence = filter_stopwords(sentence)
      list_found = check_if_list_found_in_text(sentence,FIND_WORDS)
      num_found = len(list_found)     
# Allocate a pipeline for sentiment-analysis
#nlp = pipeline('sentiment-analysis')
#nlp('We are very happy to include pipeline into the transformers repository.')
#>>> {'label': 'POSITIVE', 'score': 0.99893874}

      ss = sid.polarity_scores(sentence)
      df = pd.DataFrame.from_dict(ss,orient = "index").T
      df['text'] = sentence
      df['keywords_found'] = num_found 
      list_df.append(df)
  return pd.concat(list_df)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
#read and process data into a single dataset.  add ticker along the way.
import time

master_list_df = []
list_tickers = df_tickers['Symbol']
for ticker in list_tickers:
  
  tic = time.perf_counter()

  import os.path
  from os import path

  if path.exists("sec_data_folder/"+ticker+".csv"):

    df_text = pd.read_csv("sec_data_folder/"+ticker+".csv") 
    df_text['ticker'] = ticker
  
    print("working on..."+ticker)
  
    df_discussion = df_text[df_text['section']=='discussion']

    list_df = []
    for i,row in df_discussion.iterrows(): #this is greedy and should be vectorized with apply once debugged
      df = df_from_text(row['text']) 
      df['id'] = i 
      df['period_date'] = row['period_date']
      df['type'] = row['type']
      df['section'] = row['section']
      list_df.append(df) #this needs to be cached

    if len(list_df) > 0:
      df_sentence_text_scores = pd.concat(list_df)
      df_sentence_text_scores = df_sentence_text_scores.groupby(['period_date','type']).sum().reset_index()
      df_sentence_text_scores['ticker'] = ticker
      df_sentence_text_scores.drop(['id','keywords_found'],axis = 1)
  
    #scaled scores and differences
      df_sentence_text_scores['compound_baseline'] = df_sentence_text_scores['compound'] / df_sentence_text_scores['compound'].mean()
      df_sentence_text_scores['neg_baseline'] = df_sentence_text_scores['neg'] / df_sentence_text_scores['neg'].mean()
      df_sentence_text_scores['pos_baseline'] = df_sentence_text_scores['pos'] / df_sentence_text_scores['pos'].mean()
      df_sentence_text_scores['compound_bdiff'] = df_sentence_text_scores['compound_baseline'].diff() 
      df_sentence_text_scores['neg_bdiff'] = df_sentence_text_scores['neg_baseline'].diff() 
      df_sentence_text_scores['pos_bdiff'] = df_sentence_text_scores['pos_baseline'].diff() 
      df_sentence_text_scores['compound_zscore'] = (df_sentence_text_scores['compound'] - df_sentence_text_scores['compound'].mean())/df['compound'].std(ddof=0)

      master_list_df.append(df_sentence_text_scores)
    else:
      print("missing..."+ticker)
    toc = time.perf_counter()
    print(f"Text processed in {toc - tic:0.4f} seconds")


df_data = pd.concat(master_list_df)


working on...MMM
Text processed in 45.0568 seconds
working on...ABT
Text processed in 15.7750 seconds
working on...ABBV
Text processed in 24.1168 seconds
working on...ABMD
Text processed in 22.0094 seconds
working on...ACN
Text processed in 33.7980 seconds
working on...ATVI
Text processed in 43.2732 seconds
working on...ADBE
Text processed in 37.3221 seconds
working on...AMD
Text processed in 23.8005 seconds
working on...AAP
Text processed in 19.7655 seconds
working on...AES
Text processed in 70.9061 seconds
working on...AFL
Text processed in 75.0489 seconds
working on...A
Text processed in 36.8763 seconds
working on...APD
Text processed in 55.5121 seconds
working on...AKAM
Text processed in 36.3326 seconds
working on...ALK
Text processed in 42.1146 seconds
working on...ALB
Text processed in 47.2478 seconds
working on...ARE
Text processed in 102.7069 seconds
working on...ALXN
Text processed in 53.6985 seconds
working on...ALGN
Text processed in 34.5365 seconds
working on...ALLE
Text pr

KeyboardInterrupt: ignored

In [0]:
df_data.to_csv("df_data.csv")
drive_handler.upload('df_data.csv', parent_path='sec_data_folder',overwrite=False)

In [0]:
df = df_data[['period_date','ticker','compound_baseline']]
df['quarter_end'] = pd.to_datetime(df['period_date'])
df['quarter_end'] = df['quarter_end'].dt.to_period('q').dt.end_time
df['quarter_end'] = df['quarter_end'].dt.date
import numpy as np
df_data_pivot = pd.pivot_table(df, values='compound_baseline', index=['quarter_end'],
                    columns=['ticker'], aggfunc=np.sum, fill_value=0).reset_index()

df_data_pivot.to_csv("df_data_pivot.csv")
drive_handler.upload('df_data_pivot.csv', parent_path='sec_data_folder',overwrite=False)
print('done!')

In [0]:
df_data_pivot

In [0]:
df_data.head(6)

In [0]:
df_grouped = df_data[df_data['ticker'] == 'GOOG']
df_grouped['period_date'] = pd.to_datetime(df_grouped['period_date'])
df_grouped['period_date'] = df_grouped['period_date'].dt.strftime('%m/%d/%Y')
df_grouped.plot(x='period_date',y='compound_baseline',kind = 'bar',title = 'GOOG Baseline MD&A Sentiment NLTK')

#to do:
#debug why 6 periods
#delta baseline vs following 5 day alpha in stock
#ARIMA model fit (or sum of squares for easy explanation).
#the NLTK implementation is the base standard of comparison

#pytorch transformers sentiment analysis - language model
#as above, delta baseline vs alpha
#compare w NLTK ARIMA/linear regression fit.
